# Kaggle TPS 202109

## Optuna LightGBM Starter

### Check GPU

In [1]:
# Google Colab
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Sep 17 01:40:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Check Memory

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


### PIP Installs

Uninstall the colab default lightgbm, since it is not set up for GPU

In [4]:
!pip uninstall lightgbm


For now, git clone the lightgbm from the repo.  (There is possibly another GPU friendly install option, but it hasn't been verified to work)

Note that this will take up space, so this is being done before mounting Google Drive



In [5]:
! git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;    
# unfortunately I forgot where I got the above, so cannot give them adequate credit

Cloning into 'LightGBM'...
remote: Enumerating objects: 23457, done.
remote: Counting objects: 100% (1134/1134), done.
remote: Compressing objects: 100% (607/607), done.
remote: Total 23457 (delta 706), reused 830 (delta 507), pack-reused 22323
Receiving objects: 100% (23457/23457), 18.09 MiB | 16.88 MiB/s, done.
Resolving deltas: 100% (17099/17099), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21731, done.        
remote: Counting objects: 100% (3/3), done.        
remote: 

Other untried option

```!pip install lightgbm --install-option=--gpu```


In [6]:
# Next two lines are for Google Colab GPU and working with AutoGluon
!pip uninstall -y mkl
!pip install --upgrade mxnet-cu100

!pip install optuna

Found existing installation: mkl 2019.0
Uninstalling mkl-2019.0:
  Successfully uninstalled mkl-2019.0
     |████████████████████████████████| 352.6 MB 5.1 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 302 kB 13.2 MB/s 
     |████████████████████████████████| 80 kB 11.7 MB/s 
     |████████████████████████████████| 208 kB 74.8 MB/s 
     |████████████████████████████████| 75 kB 5.9 MB/s 
     |████████████████████████████████| 111 kB 97.6 MB/s 
     |████████████████████████████████| 49 kB 7.6 MB/s 
     |████████████████████████████████| 144 kB 98.0 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=3fad7b5750b8290b5d90610b376f23647edb4c78613049c0b588d90eebda9381
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfull

## Imports

In [39]:
import random
import os
from google.colab import output
import optuna


from lightgbm import LGBMClassifier
import lightgbm as lgb


import pandas as pd

from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import pickle
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import roc_auc_score

#importing all the plot functions
from optuna.visualization import plot_edf
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

import logging

## Google Drive Mount

* This is used to save models and output
* Note that for this Kaggle competition, the train.csv and test.csv are quite large and take a while to upload and store.

In [31]:
working_dir = 'MyDrive/kaggle/202109'
drive_dir = '/content/drive/'
model_dir = 'models'
output_dir = 'output'
from google.colab import drive
drive.mount(drive_dir, force_remount=True)  # Do this once only

mounted_drive = f'{drive_dir}/{working_dir}' #''
os.chdir(mounted_drive)


Mounted at /content/drive/


In [13]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/.kaggle"

If you want to directly download the kaggle files to your Google Colab environment

From https://www.analyticsvidhya.com/blog/2021/05/10-colab-tips-and-hacks-for-efficient-use-of-it/

https://github.com/Kaggle/kaggle-api

You can pull the datasets directly from Kaggle
1.  Go to your account page on kaggle.com and scroll to API to get your token
2.  Download the token 
3.  Copy it to Google Drive - most people put it in the .kaggle directory
4.  Set enviornment variable KAGGLE_CONFIG_DIR
5.  ```
! pip install kaggle
kaggle competitions download -c tabular-playground-series-sep-2021```
6.  You may need to add logic to move the files where you want them

In [15]:
#!pip install kaggle
#!kaggle competitions download -c tabular-playground-series-sep-2021

In [10]:
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)

# Read csv files
df = pd.read_csv('./train.csv')
test_df=pd.read_csv('./test.csv')


## Logging

In [44]:
# Right now this is not working as expected, so this is pending
#logging.basicConfig(filename='./artifacts/Kaggle_202109_lgbm.log',level=logging.INFO)

# Massaging Data

These are taken from Kaggle notebooks (see commented code)


1.   Memory optimization 
     * scans each column and based on the numeric range, sets the type to the one that requires the least memory
     * improves performance because the datasets are very large
2.   Imputation
     * uses sklearn's iterative imputer which approximates the missing values based on "nearby" values.
     * improves scores
     * note that it takes a while, so for this notebook, the massaged data is being saved locally
3.   additional columns are added to indicate the number of rows missing data and the missing feature columns

In [11]:
RUN_MASSAGE = False

In [12]:
# https://www.kaggle.com/lucamassaron/autogluon-for-tabular-playground-sep-2021
# Derived from the original script https://www.kaggle.com/gemartin/load-data-reduce-memory-usage 
# by Guillaume Martin

# This checks the range of values in each column and resets the type 

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [13]:
df = reduce_mem_usage(df)
test_df = reduce_mem_usage(test_df)

Mem. usage decreased to 435.76 Mb (50.3% reduction)
Mem. usage decreased to 224.01 Mb (50.0% reduction)


In [14]:
if RUN_MASSAGE:
  X = df.drop(columns = ['id', 'claim'])
  test_X = test_df.drop(columns = ['id'])

In [15]:
#https://www.kaggle.com/lucamassaron/autogluon-for-tabular-playground-sep-2021
# Reference: https://www.kaggle.com/hsuchialun/tps-lightgbm-kfold
if RUN_MASSAGE:
    feats = list(X.columns[:-1])

    X['n_row_missing'] = X[feats].isna().sum(axis=1)
    test_X['n_row_missing'] = test_X[feats].isna().sum(axis=1)

    X['row_std'] = X[feats].std(axis=1)
    test_X['row_std'] = test_X[feats].std(axis=1)

    feats += ['n_row_missing', 'row_std']


In [16]:
# MULTIVARIATE ITERATIVE IMPUTATION

if RUN_MASSAGE:
    
    imp = IterativeImputer(max_iter=10, random_state=0)
    imp.fit(X[feats].sample(n=10_000))

    X[feats] = imp.transform(X[feats])
    test_X[feats] = imp.transform(test_X[feats])

In [17]:
if RUN_MASSAGE:
    pd.DataFrame(X).to_csv('./reduced_mem_imputed.csv', index=False)
    pd.DataFrame(test_X).to_csv('./reduced_mem_imputesd_test.csv', index=False)

In [18]:
X = pd.read_csv('./reduced_mem_imputed.csv')
test_X = pd.read_csv('./reduced_mem_imputesd_test.csv') # this is the Kaggle test dataset

In [25]:
y = df[['claim']]
N_SPLITS = 5 # if you're using cross_score_validation
EARLY_STOPPING_ROUNDS = 400
SEED = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Optuna 
* study - this is the tuning done over several iterations of different hyperparameters
* trial - this is the individual evaluation/trial of selected hyperparameters
* objective function - defines the logic that Optuna will use for a single trial
* callback - allows actions after the objective function has returned the ranking metric to Optuna.  

At this point Optuna does not save the best model for use after the study.  Instead it will return the best parameters of the study.  However the study can take a long time, so a callback is used to save the best model (as of yet) to a file.

**Note that there should be a sample file called best_{model_type} which will be replaced by the best model of the study.  Right now the job will err out, if the model isn't there. ** (And yes, more code can be written to automatically create that file or check it's existence, but it hasn't been done yet)

Due to the large training and test datasets, the objective function has been modified to work with train_test_split

In [29]:
def lgbm_objective(trial):
   
    # hyperparameter setting, trial.suggest_uniform will suggest uniform hyperparameter
    
    
    kfolds = KFold(n_splits=N_SPLITS, shuffle=True)
    
    params = {
        "metric": "AUC",
        "verbosity": 0,
        "device": "gpu",
        "boosting_type": trial.suggest_categorical('boosting_type', ['gbdt']), # , 'dart'
        "subsample": trial.suggest_uniform('subsample', 0.5, 0.6),
        "learning_rate": trial.suggest_float('eta', 0.005, 0.015),
        "num_iterations": trial.suggest_int('num_iterations', 1000, 40000),
        "subsample_freq": 1,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-7, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-7, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),        
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    
    print(f'STARTING Params: {params}')
    lgbm_model = LGBMClassifier(**params,random_state=42)
    
    d_train = lgb.Dataset(X, label=y)

    #https://www.kaggle.com/aharless/trying-to-do-k-fold-cv-on-lightgbm
    cv_results = lgb.cv(params, d_train, early_stopping_rounds=EARLY_STOPPING_ROUNDS, nfold=N_SPLITS, metrics = 'auc', seed=42)
    
    print(cv_results)
    #results['auc-mean'][-1]
    auc = cv_results['auc-mean']
    print('Trial LGBM AUC',np.amax(auc), np.amin(auc), np.sqrt(abs(np.mean(auc))))
    filename = f'./models/lgbm_{trial.number}'
    #model = study.get_user_attr(key="best_booster", value=model)
    pickle.dump(lgbm_model, open(filename, 'wb'))
    
    return auc[-1] #np.mean(mse) #mse.mean()


def lgbm_callback(study, trial):
  # from os import remove
    filename = f'./models/lgbm_{trial.number}'
    #model = study.get_user_attr(key="best_booster", value=model)
    #pickle.dump(model, open(filename, 'wb'))
    best_model_name = './models/best_lgbm'
    if study.best_trial.number == trial.number:
        os.remove(best_model_name)
        os.rename(filename, best_model_name)
    else:
        os.remove(filename)
    print(f'filename {filename} best_model_name{best_model_name} ; best_trial_number {study.best_trial.number}')

In [32]:
lgbm_study = optuna.create_study(direction='maximize')
lgbm_study.optimize(lgbm_objective, n_trials=2, callbacks=[lgbm_callback])

[I 2021-09-17 03:30:28,882] A new study created in memory with name: no-name-b583a851-10bb-4521-ab45-70aff458ec9e
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:574: UserWarning:

Found `num_iterations` in params. Will use it instead of argument

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:578: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.



STARTING Params: {'metric': 'AUC', 'verbosity': 0, 'device': 'gpu', 'boosting_type': 'gbdt', 'subsample': 0.5046796256864662, 'learning_rate': 0.01479382263997963, 'num_iterations': 39932, 'subsample_freq': 1, 'lambda_l1': 1.070663488037652e-06, 'lambda_l2': 3.1016510474206833, 'num_leaves': 78, 'feature_fraction': 0.48808339873634976, 'min_child_samples': 35}


[I 2021-09-17 03:42:40,194] Trial 0 finished with value: 0.8150254867528088 and parameters: {'boosting_type': 'gbdt', 'subsample': 0.5046796256864662, 'eta': 0.01479382263997963, 'num_iterations': 39932, 'lambda_l1': 1.070663488037652e-06, 'lambda_l2': 3.1016510474206833, 'num_leaves': 78, 'feature_fraction': 0.48808339873634976, 'min_child_samples': 35}. Best is trial 0 with value: 0.8150254867528088.


{'auc-mean': [0.5421954431978501, 0.5536671516476975, 0.5630515687232617, 0.5678389125442559, 0.57260039063301, 0.5766984984470027, 0.7955550171001778, 0.8007122160081168, 0.8007208134861408, 0.8004930742739264, 0.8023312826392184, 0.8029361509395709, 0.8030000150246369, 0.8033486895647384, 0.8036510426126684, 0.8039044659286594, 0.8039904318045259, 0.8042314796548948, 0.8044680524514414, 0.8047038549311439, 0.8047467850741583, 0.804931165915273, 0.8049512516617586, 0.805116711223255, 0.8052791919237908, 0.805220824097743, 0.8053800894060299, 0.8054967698952863, 0.8055737642285141, 0.8056496310745773, 0.8057223393915528, 0.8057930439371674, 0.8058138456342213, 0.8058199073014467, 0.80588409420509, 0.8059491442284894, 0.8060247279459768, 0.8060936247867077, 0.806170645023452, 0.8062624691974172, 0.8062737120159829, 0.806319015961537, 0.8062902462987717, 0.806361997249321, 0.8064247780050234, 0.8064909581394755, 0.8065371231694447, 0.8065962763849772, 0.8066263101347217, 0.80666996768539

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:574: UserWarning:

Found `num_iterations` in params. Will use it instead of argument

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:578: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.

[I 2021-09-17 04:09:08,012] Trial 1 finished with value: 0.8158820917324716 and parameters: {'boosting_type': 'gbdt', 'subsample': 0.5867393181342582, 'eta': 0.008335091542167678, 'num_iterations': 32096, 'lambda_l1': 1.5907670641234665e-06, 'lambda_l2': 0.18248134209444805, 'num_leaves': 35, 'feature_fraction': 0.46603938972650105, 'min_child_samples': 99}. Best is trial 1 with value: 0.8158820917324716.


{'auc-mean': [0.5400453025049932, 0.5511572640541071, 0.5597843954242432, 0.5645282096782973, 0.5693626284170643, 0.5728354828634228, 0.7974650289877628, 0.7964659325314724, 0.7959601154731589, 0.7947244738339666, 0.8006896823010632, 0.8023552670664842, 0.8022805273605929, 0.8028631009760874, 0.8031033132845463, 0.8032993555295185, 0.8033574464496374, 0.8035607659558789, 0.8037965656596839, 0.8040274920237713, 0.804067129491431, 0.8042806072236784, 0.8042440194315406, 0.8044456392057281, 0.8046178168337212, 0.804633571458233, 0.8048272424444342, 0.8049577416059279, 0.8048048873210798, 0.8049196492557298, 0.8050308388809903, 0.8050865014202813, 0.8051110329539661, 0.8050701496370941, 0.8051605338204814, 0.8052393576098611, 0.805257783923351, 0.8053095770758192, 0.8053621658047432, 0.80540734687797, 0.8053756329237428, 0.8053442220259968, 0.8052888468558483, 0.8052877142099147, 0.8053709724117363, 0.805420476263721, 0.8054896544612273, 0.8055488363182945, 0.8055496481261896, 0.8055953887

In [50]:
 # To get the dictionary of parameter name and parameter values:
 # Right now logging is not working as expected so this is pending
#logging.info(f'Return a dictionary of parameter name and parameter values:{lgbm_study.best_params}')

results = []
print(f'Return a dictionary of parameter name and parameter values:{lgbm_study.best_params}')
results.append({'msg':lgbm_study.best_params})
 
# To get the best observed value of the objective function:
#logging.info(f'Return the best observed value of the objective function: {lgbm_study.best_value}')
print(f'Return the best observed value of the objective function: {lgbm_study.best_value}')
results.append({'msg':lgbm_study.best_value})
 
# To get the best trial:
print(f'Return the best trial: {lgbm_study.best_trial}')
results.append({'msg': lgbm_study.best_trial})

pd.DataFrame(results).to_csv('results.csv', index=False)
 
# To get all trials:
#print("Return all the trials:", study.trials)

Return a dictionary of parameter name and parameter values:{'boosting_type': 'gbdt', 'subsample': 0.5867393181342582, 'eta': 0.008335091542167678, 'num_iterations': 32096, 'lambda_l1': 1.5907670641234665e-06, 'lambda_l2': 0.18248134209444805, 'num_leaves': 35, 'feature_fraction': 0.46603938972650105, 'min_child_samples': 99}
Return the best observed value of the objective function: 0.8158820917324716
Return the best trial: FrozenTrial(number=1, values=[0.8158820917324716], datetime_start=datetime.datetime(2021, 9, 17, 3, 42, 40, 200646), datetime_complete=datetime.datetime(2021, 9, 17, 4, 9, 8, 11937), params={'boosting_type': 'gbdt', 'subsample': 0.5867393181342582, 'eta': 0.008335091542167678, 'num_iterations': 32096, 'lambda_l1': 1.5907670641234665e-06, 'lambda_l2': 0.18248134209444805, 'num_leaves': 35, 'feature_fraction': 0.46603938972650105, 'min_child_samples': 99}, distributions={'boosting_type': CategoricalDistribution(choices=('gbdt',)), 'subsample': UniformDistribution(high=

In [33]:
model_file = open('./models/best_lgbm', 'rb')
colab_lgbm_model_eval = pickle.load(model_file)
model_file.close()

In [34]:
colab_lgbm_model_eval.fit(X,y)
y_predproba = colab_lgbm_model_eval.predict_proba(X)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:177: UserWarning:

Found `num_iterations` in params. Will use it instead of argument



[LightGBM] [Warning] feature_fraction is set=0.46603938972650105, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.46603938972650105
[LightGBM] [Warning] lambda_l2 is set=0.18248134209444805, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.18248134209444805
[LightGBM] [Warning] lambda_l1 is set=1.5907670641234665e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.5907670641234665e-06
[LightGBM] [Warning] feature_fraction is set=0.46603938972650105, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.46603938972650105
[LightGBM] [Warning] lambda_l2 is set=0.18248134209444805, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.18248134209444805
[LightGBM] [Warning] lambda_l1 is set=1.5907670641234665e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.5907670641234665e-06


In [35]:
roc_auc_score(y, y_predproba[:, 1])

0.9314746128598799

In [36]:
y_predproba = pd.DataFrame(colab_lgbm_model_eval.predict_proba(test_X))
y_predproba['id'] = y_predproba.index + 957919
y_predproba.to_csv('./artifacts/lightgbm_predproba.csv', index=False)
results_df = y_predproba.rename(columns={1:'claim'}).copy()
results_df = results_df[['id','claim']]
results_df.to_csv('./artifacts/lightgbm_submission.csv', index=False)